# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [1]:
import pandas as pd
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0)
recipes = pd.read_csv('data/recipes_sample.csv', index_col=0, usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])
print(reviews.head())
print(recipes.head())

           user_id  recipe_id        date  rating  \
370476       21752      57993  2003-05-01       5   
624300      431813     142201  2007-09-16       5   
187037      400708     252013  2008-01-10       4   
706134  2001852463     404716  2017-12-11       5   
312179       95810     129396  2008-03-14       5   

                                                   review  
370476  Last week whole sides of frozen salmon fillet ...  
624300  So simple and so tasty!  I used a yellow capsi...  
187037  Very nice breakfast HH, easy to make and yummy...  
706134  These are a favorite for the holidays and so e...  
312179  Excellent soup!  The tomato flavor is just gre...  
                                             id  minutes   submitted  \
name                                                                   
george s at the cove  black bean soup     44123       90  2002-10-25   
healthy for them  yogurt popsicles        67664       10  2003-07-26   
i can t believe it s spinach     

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [2]:
reviews_sample = reviews.sample(frac=0.05)
recipes_sample = recipes.sample(frac=0.05)

with pd.ExcelWriter('recipes.xlsx') as writer:
    reviews_sample.to_excel(writer, sheet_name='Отзывы')
    recipes_sample.to_excel(writer, sheet_name='Рецепты')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [3]:
import pandas as pd
import xlwings as xw
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
recipes = sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value
recipes['seconds_assign'] = recipes['minutes'] * 60
sheet.range('G1').options(transpose=True).value = recipes['seconds_assign'].values
wb.save()
wb.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [5]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet.range('H1').value = 'seconds_formula'
last_row = sheet.range('A1').current_region.last_cell.row
sheet.range('H2:H{}'.format(last_row)).formula = '=B2*60'
wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [6]:
import xlwings as xw
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
headers = sheet.range('A1:H1')
headers.api.Font.Bold = True
headers.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
wb.save()
wb.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [11]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
green = (50, 205, 50)
yellow = (255, 255, 0)
red = (255, 99, 71)
last_row = sheet.range('A1').current_region.last_cell.row
for i in range(2, last_row + 1):
    minutes = sheet.range('C{}'.format(i)).value
    if minutes < 5:
        color = green
    elif minutes < 10:
        color = yellow
    else:
        color = red
    sheet.range('E{}'.format(i)).color = color
wb.save()
wb.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [12]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet.range('I1').value = 'n_reviews'
last_row = sheet.range('A1').current_region.last_cell.row
sheet.range('I2:I{}'.format(last_row)).formula = '=COUNTIF(Отзывы!A:A, A2)'
wb.save()
wb.close()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [31]:
def validate():
    wb = xw.Book('recipes.xlsx')
    recipes_sheet = wb.sheets['Рецепты']
    reviews_sheet = wb.sheets['Отзывы']
    red = (255, 0, 0)
    last_row = reviews_sheet.range('A1').current_region.last_cell.row
    range_str = 'A2:C{}'.format(last_row)
    range = reviews_sheet.range(range_str)
    for row in range.rows:
        rating = row[1].value
        if rating is None or not (0 <= rating <= 5):
            row.color = red
            continue
        
        recipe_id = row[2].value
        if recipe_id is None or recipes_sheet.range('A:A').find(recipe_id) is None:
            row.color = red
            continue
    wb.save()
    wb.close()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [37]:
import csv
with open('data/recipes_model.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
wb = xw.Book()
wb.sheets[0].name = 'Модель'
sheet = wb.sheets['Модель']
sheet.range('A1').value = data
wb.save('recipes_model.xlsx')
wb.close()

![](1.png)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [7]:
import xlwings as xw

wb = xw.Book('recipes_model.xlsx')

sheet = wb.sheets['Модель']
header_format = sheet.range('A1').expand('right')
header_format.Color = 0x00ccff
header_format.api.Font.Bold = True
sheet.api.Cells.EntireColumn.AutoFit()
sheet.range('A1').expand('right').api.AutoFilter(Field:=1)

wb.save()
wb.close()

![](2.png)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [6]:
import pandas as pd
import xlwings as xw

df = pd.read_csv('data/recipes_model.csv', delimiter='\t')
stats = df.groupby('recipe').size()
wb = xw.Book()
wb.sheets[0].name = 'Статистика'
sheet = wb.sheets['Статистика']
sheet.range('A1').value = stats.to_frame()
chart = sheet.charts.add()
chart.set_source_data(sheet.range('A1').expand())
chart.chart_type = 'column_clustered'
chart.name = 'Количество атрибутов'
chart.top = sheet.range('E2').top
chart.left = sheet.range('E2').left

wb.save('recipes_stats.xlsx')
wb.close()

![](3.png)